In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import username
from config import password
from config import host_number

In [2]:
# https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index.aspx
csv_file = "data/hpi_data.csv"
hpi_data_df = pd.read_csv(csv_file)
hpi_data_df.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98,101.06
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33,100.95
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72,101.02
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35,101.40


In [3]:
#drop columns
hpi_data_clean = hpi_data_df[["hpi_type","hpi_flavor","frequency","level","place_name","place_id","yr","period","index_nsa"]]
# hpi_data_clean.head()

In [4]:
#level = location
# index nsa = price
#yr = year
#hpi flavor = purchase type
hpi_df = hpi_data_clean.rename(columns = {"level": "location", "index_nsa": "price", "yr":"year", "hpi_flavor":"purchase_type"})
hpi_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35


In [5]:
#state dataframe (quarterly)
state_df = hpi_df.loc[hpi_df["location"] == "State"]
state_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
64331,traditional,all-transactions,quarterly,State,Alaska,AK,1975,1,62.74
64332,traditional,all-transactions,quarterly,State,Alaska,AK,1975,2,64.16
64333,traditional,all-transactions,quarterly,State,Alaska,AK,1975,3,62.24
64334,traditional,all-transactions,quarterly,State,Alaska,AK,1975,4,68.51
64335,traditional,all-transactions,quarterly,State,Alaska,AK,1976,1,71.31


In [6]:
#usa dataframe (monthly)
usa_df = hpi_df.loc[hpi_df["location"] == "USA or Census Division"]
usa_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.98
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.33
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.72
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.35


In [8]:
#puerto rico dataframe (quarterly)
pr_df = hpi_df.loc[hpi_df["location"] == "Puerto Rico"]
pr_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
114523,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,1,100.00
114524,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,2,102.34
114525,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,3,106.22
114526,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1995,4,111.45
114527,developmental,all-transactions,quarterly,Puerto Rico,Puerto Rico,PR,1996,1,113.46


In [6]:
#sort state data to get period, years grouped together
sorted_state_df = state_df.sort_values(["year","period"])
sorted_state_df.head()

,hpi_type,purchase_type,frequency,location,place_name,place_id,year,period,price
64331,traditional,all-transactions,quarterly,State,Alaska,AK,1975,1,62.74
64515,traditional,all-transactions,quarterly,State,Alabama,AL,1975,1,72.93
64699,traditional,all-transactions,quarterly,State,Arkansas,AR,1975,1,66.58
64883,traditional,all-transactions,quarterly,State,Arizona,AZ,1975,1,59.65
65067,traditional,all-transactions,quarterly,State,California,CA,1975,1,41.68


In [7]:
#DF to Isolate Period
state_period1 = state_df.loc[state_df['period'] == 1]
state_period4 = state_df.loc[state_df['period'] == 4]
state_period4
#State Data Renamed
state_period1 = state_period1.rename(columns={"period": "period1", "price": "price1"})
state_period1
state_period4 = state_period4.rename(columns={"period": "period4", "price": "price4"})
state_period4
#Create dataframes with beginning and ending prices
beginning_price = state_period1[["place_name", "place_id", "year", "period1", "price1"]]
beginning_price
ending_price = state_period4[["period4", "price4"]]
ending_price
#Merge Data
state_merged = pd.concat([beginning_price.reset_index(drop=True),ending_price.reset_index(drop=True)], axis=1)
state_merged
state_merged["yearly_change"] = (state_merged["price4"] - state_merged['price1']) * 100 / state_merged['price1']
state_merged
max_state = state_merged.loc[state_merged['yearly_change'].idxmax()]
max_state
min_state = state_merged.loc[state_merged['yearly_change'].idxmin()]
min_state
print(max_state)
print(min_state)
beginning_price
state_merged

place_name       South Dakota
place_id                   SD
year                     1982
period1                     1
price1                  68.84
period4                     4
price4                 119.08
yearly_change         72.9808
Name: 1893, dtype: object
place_name        Hawaii
place_id              HI
year                1981
period1                1
price1            102.41
period4                4
price4              49.4
yearly_change   -51.7625
Name: 512, dtype: object


,place_name,place_id,year,period1,price1,period4,price4,yearly_change
0,Alaska,AK,1975,1,62.74,4,68.51,9.196685
1,Alaska,AK,1976,1,71.31,4,73.68,3.323517
2,Alaska,AK,1977,1,78.73,4,73.91,-6.122190
3,Alaska,AK,1978,1,79.88,4,88.10,10.290436
4,Alaska,AK,1979,1,90.34,4,96.60,6.929378
...,...,...,...,...,...,...,...,...
6623,Wyoming,WY,2016,1,238.51,4,241.40,1.211689
6624,Wyoming,WY,2017,1,240.06,4,244.89,2.011997
6625,Wyoming,WY,2018,1,251.14,4,254.07,1.166680
6626,Wyoming,WY,2019,1,253.56,4,266.80,5.221644


In [8]:
engine = create_engine("sqlite:///housingdata.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
print(Base.classes.keys())
#Save reference to the tables
state = Base.classes.state
usa = Base.classes.usa
puerto_rico = Base.classes.puerto_rico
quarterly_states = Base.classes.quarterly_states
state_percent_increase = Base.classes.state_percent_increase

['puerto_rico', 'quarterly_states', 'state', 'state_percent_increase', 'usa']


In [65]:
state_df.to_sql(name='state', con=engine, if_exists='append', index=False)

In [66]:
pr_df.to_sql(name='puerto_rico', con=engine, if_exists='append', index=False)

In [67]:
usa_df.to_sql(name='usa', con=engine, if_exists='append', index=False)

In [10]:
state_merged.to_sql(name='state_percent_increase', con=engine, if_exists='append', index=False)

In [69]:
sorted_state_df.to_sql(name='quarterly_states', con=engine, if_exists='append', index=False)

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: INSERT INTO quarterly_states (hpi_type, purchase_type, frequency, location, place_name, place_id, year, period, price) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('traditional', 'all-transactions', 'quarterly', 'State', 'Alaska', 'AK', 1975, 1, 62.74), ('traditional', 'all-transactions', 'quarterly', 'State', 'Alabama', 'AL', 1975, 1, 72.93), ('traditional', 'all-transactions', 'quarterly', 'State', 'Arkansas', 'AR', 1975, 1, 66.58), ('traditional', 'all-transactions', 'quarterly', 'State', 'Arizona', 'AZ', 1975, 1, 59.65), ('traditional', 'all-transactions', 'quarterly', 'State', 'California', 'CA', 1975, 1, 41.68), ('traditional', 'all-transactions', 'quarterly', 'State', 'Colorado', 'CO', 1975, 1, 54.36), ('traditional', 'all-transactions', 'quarterly', 'State', 'Connecticut', 'CT', 1975, 1, 62.2), ('traditional', 'all-transactions', 'quarterly', 'State', 'District of Columbia', 'DC', 1975, 1, 46.37)  ... d